In [1]:
import pandas as pd
import random
#import torch
#from transformers import AutoTokenizer, AutoModel
#import re
#import string
import numpy as np
#import pydot
#from nltk.corpus import stopwords 
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import LancasterStemmer
#from simpletransformers.classification import MultiLabelClassificationModel
#import logging
#import custom_sentence_tokenizer
import matplotlib.pyplot as plt
#from scipy import stats
#from ast import literal_eval
import pickle
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K
import keras
import gc
from keras.models import model_from_json

Using TensorFlow backend.


In [35]:
#from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, Reshape
from keras.layers import GlobalMaxPooling1D,Conv2D, Conv1D, AveragePooling2D, MaxPooling1D, AveragePooling1D, LeakyReLU
from keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import concatenate
#from sklearn.decomposition import PCA
import _pickle as cPickle
import io
import time
import sys
import boto3
from boto3.s3.transfer import TransferConfig
import tensorflow as tf

## 1. Import Embeddings (X) and Labels (Y)

In [3]:
now = time.time()
infile = None
infile = open("embeddings_evaluation",'rb')
embeddings_evaluation = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

44.87386226654053


0

In [4]:
now = time.time()
infile = None
infile = open("embeddings_7.1",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

45.81072735786438


0

In [ ]:
embeddings.update(embeddings_evaluation)
pickle.dump( embeddings, open( "/home/ec2-user/SageMaker/embeddings.p", "wb" ) )

In [ ]:
del(embeddings_evaluation)

In [3]:
now = time.time()
infile = None
infile = open("embeddings.p",'rb')
embeddings = cPickle.load(infile)
infile.close()
del(infile)
print(time.time()-now)
gc.collect()

70.81700110435486


0

In [4]:
embeddings_np_complete = np.array(list(embeddings.items()))
del(embeddings)

## 2. Load chapter model and helper functions

In [5]:
#Load chapeter model weights
chapter_model = None
json_file = open('model_num_cnn2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
chapter_model = model_from_json(loaded_model_json)
opt = keras.optimizers.Adam(lr=0.001)
# load weights into new model
chapter_model.load_weights("model_num_cnn2.h5")
chapter_model.trainable = False

In [105]:
def padding(embeddings):
    max_sentence = 20
    padding = max_sentence - len(embeddings)
    if padding > 0:
        padding_shape = (padding, 256, 768)
        pad = np.zeros(padding*256*768).reshape(padding_shape)
        return np.append(embeddings, pad, axis = 0).astype('float16')
    else:
        return embeddings
    
def split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat):
    HADM_ID_DF = None
    loaded_model = None
    HADM_ID_DF =  pd.DataFrame(data=HADM_ID_LIST, columns=['HADM_ID'])
    CHAPTER_PIVOT_DF_COPY = HADM_ID_DF.set_index('HADM_ID')\
                    .join(CHAPTER_PIVOT_DF, how='left').copy(deep=False)
    CHAPTER_PIVOT_NP = np.array(CHAPTER_PIVOT_DF_COPY)
    return train_test_split(embeddings_concat, CHAPTER_PIVOT_NP, test_size=0.2)

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

import tensorflow as tf
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    
    #p1 = tn / (tn + fn + K.epsilon())
    #r1 = tn / (tn + fp + K.epsilon())
    
    f1 = p*r / (p+r+K.epsilon())
    #+ p1*r1 / (p1+r1+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def get_model(label_len):
    #define model - STATIC LAYERS
    model = Sequential()
    for layer in chapter_model.layers[:4]:
        layer.trainable = False
        model.add(layer)
    
    #independent_model = True
    #LEARNING LAYERS
    if label_len <=4:
        print('model 1')
        model.add(Dropout(0.5))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.5))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.5))
        model.add(Dense(label_len, activation='sigmoid'))
    elif label_len >4:
        print('model 2')
        #model.add(Conv1D(label_len, 4))
        #model.add(AveragePooling1D(pool_size=2))
        #model.add(Flatten())
        model.add(Dropout(0.3))
        model.add(Dense(2048))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.3))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.3))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(0.3))
        model.add(Dense(128))
        model.add(LeakyReLU(alpha=0.3))
        #model.add(Dropout(0.3))
        model.add(Dense(label_len, activation='sigmoid'))
    elif independent_model:
        print('model 3')
        deep_inputs = Input(shape=(20*256,768))
        LSTM_Layer_1 = LSTM(128)(deep_inputs)
        dense_layer_1 = Dense(label_len, activation='sigmoid')(LSTM_Layer_1)
        model = Model(inputs=deep_inputs, outputs=dense_layer_1)
        return model
    model = Model(inputs=model.input, outputs=model.output)
    return model

## 3. Loop on individual therapy area (bag of models)

In [46]:
score_list = []
history_list = []
#file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','280_289.csv','290_319.csv','320_389.csv','390_459.csv'
#,'460_519.csv','520_579.csv','580_629.csv','630_679__740_759__760_779.csv', '710-739.csv','780-799.csv','800-999.csv','E_V.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
file_name_LIST = ['320_389.csv']
#file_name_LIST = ['630_679__740_759__760_779.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    



============Started 320_389.csv============
320_389.csv orginal shape: (3834, 2)
320_389.csv Padding complete
320_389.csv modified shape: (3834, 5120, 768)
Train and Test Shapre: (3067, 5120, 768), (767, 5120, 768), (3067, 4), (767, 4)


In [95]:
class_value = ((y_train2.shape[0]*y_train2.shape[1])/np.sum(y_train2, axis=0))*(max(np.sum(y_train2, axis = 0))*(1/np.sum(y_train2, axis = 0)))
class_weight = {}
for i, class_value_item in enumerate(class_value):
    class_weight[i] = class_value_item
    print(i)

0
1
2
3


In [96]:
class_weight

{0: 11.412093023255814,
 1: 35.55884502037041,
 2: 49.14990831979249,
 3: 14.46023957676599}

In [82]:
((y_train2.shape[0]*y_train2.shape[1])/np.sum(y_train2, axis=0))*(max(np.sum(y_train2, axis = 0))*(1/np.sum(y_train2, axis = 0)))

array([11.41209302, 35.55884502, 49.14990832, 14.46023958])

In [106]:
    #call model
    label_len = y_train2.shape[1]
    model = get_model(label_len)
    opt = keras.optimizers.Adam(lr=0.0001)
    #'binary_crossentropy'
    
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    history = model.fit(X_train2, y_train2, epochs=5,batch_size=32, verbose=1, validation_data = (X_val,y_val))
    #history = model.fit(data_generator(X_train2, y_train2, BS, aug_num), steps_per_epoch = SPE, epochs=10, verbose=1, class_weight=class_weight)
    #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')

model 1
Train on 3067 samples, validate on 191 samples
Epoch 1/5
3067/3067 [==============================] - 69s 22ms/step - loss: 0.7598 - get_f1: 0.3291 - accuracy: 0.6810 - val_loss: 0.5255 - val_get_f1: 0.2694 - val_accuracy: 0.7513
Epoch 2/5
3067/3067 [==============================] - 68s 22ms/step - loss: 0.5712 - get_f1: 0.4395 - accuracy: 0.7430 - val_loss: 0.5159 - val_get_f1: 0.2927 - val_accuracy: 0.7448
Epoch 3/5
3067/3067 [==============================] - 68s 22ms/step - loss: 0.5001 - get_f1: 0.4884 - accuracy: 0.7680 - val_loss: 0.5137 - val_get_f1: 0.3981 - val_accuracy: 0.7461
Epoch 4/5
3067/3067 [==============================] - 67s 22ms/step - loss: 0.4577 - get_f1: 0.5466 - accuracy: 0.7912 - val_loss: 0.5251 - val_get_f1: 0.4191 - val_accuracy: 0.7317
Epoch 5/5
3067/3067 [==============================] - 67s 22ms/step - loss: 0.4188 - get_f1: 0.6043 - accuracy: 0.8138 - val_loss: 0.5498 - val_get_f1: 0.3519 - val_accuracy: 0.7408
Model evaluation 320_389.csv
7

In [24]:
aug_num = 3
BS = 10
#steps_per_epoch
SPE = (len(y_train2)*aug_num)/((1+aug_num)*BS)

def data_generator(X_train2, y_train2, BS, aug_num):
    N = len(y_train2)
    X_yield = []
    y_yield = []
    pad = [[0 for i in range(768)] for j in range(20*256)]
    #pad = np.zeros(20*256*768).reshape(20*256, 768)
    rand_LIST = np.array([i for i in range(20*256)])
    
    while True:

        for i in range(N):
            x = X_train2[i]
            y = y_train2[i]
            non_zero_x = sum(x[:,0]!=0)

            x_list = [x]
            for aug in range(aug_num):
                
                rand_NEW = rand_LIST[:non_zero_x]
                random.shuffle(rand_NEW)
                rand_NEW = np.append(rand_NEW,[n for n in range(non_zero_x, 20*256)], axis=0).astype('int')
                #print(rand_NEW)
                x_list.append([x[n] for n in rand_NEW.tolist()])

            y_list = [y for n in range(len(x_list))]
            
            if (i+1)%(BS+1) == 0:
                yield (np.array(X_yield), np.array(Y_yield))
            elif i%(BS+1) == 0:
                X_yield = x_list
                Y_yield = y_list
            else:
                X_yield = np.append(X_yield, x_list, axis=0).astype('float16')
                Y_yield = np.append(Y_yield, y_list, axis=0).astype('int') 
            

In [58]:
score_list = []
history_list = []
file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','280_289.csv','290_319.csv','320_389.csv','390_459.csv'
,'460_519.csv','520_579.csv','580_629.csv','630_679__740_759__760_779.csv', '710-739.csv','780-799.csv','800-999.csv','E_V.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['240_279_1.csv']
#file_name_LIST = ['630_679__740_759__760_779.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    
    #call model
    label_len = y_train2.shape[1]
    model = get_model(label_len)
    opt = keras.optimizers.Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    history = model.fit(X_train2, y_train2, epochs=4,batch_size=32, verbose=1, validation_data = (X_val,y_val))
    
    #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')



============Started 001_139.csv============
001_139.csv orginal shape: (5135, 2)
001_139.csv Padding complete
001_139.csv modified shape: (5135, 5120, 768)
Train and Test Shapre: (4108, 5120, 768), (1027, 5120, 768), (4108, 4), (1027, 4)
model 1
Train on 4108 samples, validate on 256 samples
Epoch 1/4
4108/4108 [==============================] - 86s 21ms/step - loss: 0.5392 - get_f1: 0.5945 - accuracy: 0.7622 - val_loss: 0.5104 - val_get_f1: 0.5871 - val_accuracy: 0.7695
Epoch 2/4
4108/4108 [==============================] - 86s 21ms/step - loss: 0.4600 - get_f1: 0.6467 - accuracy: 0.8003 - val_loss: 0.5071 - val_get_f1: 0.6013 - val_accuracy: 0.7725
Epoch 3/4
4108/4108 [==============================] - 86s 21ms/step - loss: 0.4150 - get_f1: 0.6810 - accuracy: 0.8169 - val_loss: 0.5055 - val_get_f1: 0.5986 - val_accuracy: 0.7725
Epoch 4/4
4108/4108 [==============================] - 86s 21ms/step - loss: 0.3651 - get_f1: 0.7314 - accuracy: 0.8431 - val_loss: 0.5160 - val_get_f1: 0.6

In [59]:
#'001_139.csv','140_239.csv','240_279.csv','800-999.csv'
# Run 800-999 and 240-279 with more epochs: 6 and 10
pickle.dump( score_list, open( "/home/ec2-user/SageMaker/score_list.p", "wb" ) )
pickle.dump( history_list, open( "/home/ec2-user/SageMaker/history_list.p", "wb" ) )

In [16]:
import tensorflow as tf
print(tf.__version__)

2.0.2


In [60]:
score_list = []
history_list = []
file_name_LIST = ['460_519.csv','520_579.csv','580_629.csv','630_679__740_759__760_779.csv', '710-739.csv','780-799.csv','800-999.csv','E_V.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['240_279_1.csv']
#file_name_LIST = ['630_679__740_759__760_779.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    
    #call model
    label_len = y_train2.shape[1]
    model = get_model(label_len)
    opt = keras.optimizers.Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    history = model.fit(X_train2, y_train2, epochs=4,batch_size=32, verbose=1, validation_data = (X_val,y_val))
    
    #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')



============Started 460_519.csv============
460_519.csv orginal shape: (9021, 2)
460_519.csv Padding complete
460_519.csv modified shape: (9021, 5120, 768)
Train and Test Shapre: (7216, 5120, 768), (1805, 5120, 768), (7216, 5), (1805, 5)
model 2
Train on 7216 samples, validate on 451 samples
Epoch 1/4
7216/7216 [==============================] - 179s 25ms/step - loss: 0.5764 - get_f1: 0.3256 - accuracy: 0.7073 - val_loss: 0.5286 - val_get_f1: 0.2915 - val_accuracy: 0.7446
Epoch 2/4
7216/7216 [==============================] - 208s 29ms/step - loss: 0.5262 - get_f1: 0.4101 - accuracy: 0.7417 - val_loss: 0.5229 - val_get_f1: 0.3737 - val_accuracy: 0.7534
Epoch 3/4
7216/7216 [==============================] - 209s 29ms/step - loss: 0.4975 - get_f1: 0.4626 - accuracy: 0.7583 - val_loss: 0.5223 - val_get_f1: 0.3964 - val_accuracy: 0.7548
Epoch 4/4
7216/7216 [==============================] - 209s 29ms/step - loss: 0.4657 - get_f1: 0.5165 - accuracy: 0.7772 - val_loss: 0.5239 - val_get_f1:

In [62]:
score_list = []
history_list = []
file_name_LIST = ['adverse_effect.csv']
1. #file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','800-999.csv']
#file_name_LIST = ['240_279_1.csv']
#file_name_LIST = ['630_679__740_759__760_779.csv']

for file_name in file_name_LIST:
    print('\n\n============Started '+file_name+'============')
    CHAPTER_PIVOT_DF = pd.read_csv('Layer-2/'+file_name, sep=',', header = 0)\
                        .astype({'HADM_ID': 'str'}).set_index('HADM_ID')
    HADM_ID_LIST = list(CHAPTER_PIVOT_DF.index)
    embeddings_np = embeddings_np_complete[np.isin(embeddings_np_complete[:,0], HADM_ID_LIST)]
    print(file_name+' orginal shape: '+str(embeddings_np.shape))
    HADM_ID_LIST = embeddings_np[:,0].copy()
    embeddings = embeddings_np[:,1].copy()
    
    #Zero Pad embeddings
    embeddings_padded = [padding(embeddings_ITEM) for embeddings_ITEM in embeddings]
    del(embeddings)
    gc.collect()
    print(file_name+' Padding complete')

    #Reshape embeddings
    #embeddings_concat = np.array([np.concatenate(i) for i in embeddings_padded])
    embeddings_concat = np.array(embeddings_padded).reshape(len(embeddings_padded), 20*256, 768)
    print(file_name+' modified shape: '+str(embeddings_concat.shape))
    del(embeddings_padded)
    gc.collect()
    
    X_train2, X_test2, y_train2, y_test2 = split_for_train(HADM_ID_LIST, CHAPTER_PIVOT_DF, embeddings_concat)
    print('Train and Test Shapre: '+str(X_train2.shape)+', '+str(X_test2.shape)+', '+str(y_train2.shape)+', '+str(y_test2.shape))
    X_val = X_test2[:int(0.25*len(X_test2))]
    y_val = y_test2[:int(0.25*len(y_test2))]
    del(HADM_ID_LIST)
    del(CHAPTER_PIVOT_DF)
    del(embeddings_concat)
    gc.collect()
    
    #call model
    label_len = y_train2.shape[1]
    model = get_model(label_len)
    opt = keras.optimizers.Adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[get_f1, 'accuracy'])
    history = model.fit(X_train2, y_train2, epochs=4,batch_size=32, verbose=1, validation_data = (X_val,y_val))
    
    #evaluate model
    print('Model evaluation '+file_name)
    score = model.evaluate(X_test2, y_test2, verbose=1)
    print(score)
    
    #save model evaluation and history metrics
    score_list.append(score)
    history_list.append(history.history)
    
    #save model
    model_json = model.to_json()
    with open("/home/ec2-user/SageMaker/Models/model_"+file_name+".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights("/home/ec2-user/SageMaker/Models/model_"+file_name+".h5")
    print('============completed '+file_name+'============')



============Started adverse_effect.csv============
adverse_effect.csv orginal shape: (6339, 2)
adverse_effect.csv Padding complete
adverse_effect.csv modified shape: (6339, 5120, 768)
Train and Test Shapre: (5071, 5120, 768), (1268, 5120, 768), (5071, 5), (1268, 5)
model 2
Train on 5071 samples, validate on 317 samples
Epoch 1/4
5071/5071 [==============================] - 93s 18ms/step - loss: 0.6137 - get_f1: 0.2469 - accuracy: 0.6700 - val_loss: 0.5725 - val_get_f1: 0.1195 - val_accuracy: 0.7054
Epoch 2/4
5071/5071 [==============================] - 98s 19ms/step - loss: 0.5610 - get_f1: 0.3124 - accuracy: 0.7078 - val_loss: 0.5533 - val_get_f1: 0.3006 - val_accuracy: 0.7091
Epoch 3/4
5071/5071 [==============================] - 98s 19ms/step - loss: 0.5343 - get_f1: 0.3958 - accuracy: 0.7283 - val_loss: 0.5553 - val_get_f1: 0.3046 - val_accuracy: 0.7136
Epoch 4/4
5071/5071 [==============================] - 98s 19ms/step - loss: 0.5076 - get_f1: 0.4448 - accuracy: 0.7428 - val_lo

In [65]:
pwd

'/home/ec2-user/SageMaker'

In [63]:
file_name_LIST = ['001_139.csv','140_239.csv','240_279.csv','280_289.csv','290_319.csv','320_389.csv','390_459.csv'
,'460_519.csv','520_579.csv','580_629.csv','630_679__740_759__760_779.csv', '710-739.csv','780-799.csv','800-999.csv','E_V.csv']

In [66]:
#Load chapeter model weights
for file_name in file_name_LIST:
    chapter_model = None
    json_file = open('/home/ec2-user/SageMaker/Models/model_'+file_name+'.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    chapter_model = model_from_json(loaded_model_json)
    opt = keras.optimizers.Adam(lr=0.001)
    # load weights into new model
    chapter_model.load_weights('/home/ec2-user/SageMaker/Models/model_'+file_name+'.h5')
    chapter_model.summary()
    

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5120, 768)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5113, 16)          98320     
_________________________________________________________________
average_pooling1d_4 (Average (None, 1278, 16)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 20448)             0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 20448)             0         
_________________________________________________________________
dense_45 (Dense)             (None, 128)               2617472   
_________________________________________________________________
dropout_22 (Dropout)         (None, 128)               0  